In [1]:
import warnings
warnings.filterwarnings('ignore')

import time
import pandas as pd
import numpy as np

import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
df = pd.read_csv("C:/Users/lauta/Desktop/Internship DealRoom/data_scientist_intern_G2_scraper.csv")

In [3]:
df.head()

,id,NAME,WEBSITE,TAGLINE,ADDRESS,INDUSTRIES
0,1773270,Matera (formerly illiCopros),https://www.matera.eu/,Allows residential property owners to self-man...,"12 Rue du 4 septembre, 75002 Paris, France",real estate
1,1659999,Shift,https://shift.online/,"Move Anything, Anywhere, Anytime","40 Islington High St, Islington, London N1",travel;transportation
2,1617238,MILES Mobility,https://miles-mobility.com/,An independent carsharing and modern mobility,"Leibnizstraße 49, 10629 Berlin, Germany",transportation
3,1519744,Doktor24,https://doktor24.se,Leading the digitalization of Swedish primary ...,"Birger Jarlsgatan, Stockholm, Sweden",health
4,975908,Forecast,https://www.forecast.app,AI-powered resource & project management platform,"20, Frederiksborggade, 1360 Copenhagen, Denmark",enterprise software


#### Clean names of the companies to use them as search queries

In [4]:
no_parenthesis = []
no_commas = []
for name in df.NAME[df.NAME.str.contains('(', regex=False)].str.split('('):
    no_parenthesis.append(name[0])
    
for name in df.NAME[df.NAME.str.contains(',', regex=False)].str.split(','):
    no_commas.append(name[0])

In [5]:
df.NAME[df.NAME.str.contains('(', regex=False)] = no_parenthesis
df.NAME[df.NAME.str.contains(',', regex=False)] = no_commas
df.NAME = df.NAME.str.strip()
df.NAME = df.NAME.str.replace(' ', '+')

#### G2 Scraper

In [6]:
options = webdriver.ChromeOptions() 
options.add_argument("start-maximized")

In [7]:
g2_webs = []
ratings = []
reviews = []
product_descriptions = []
alternatives = []

#Initialize chrome bot
driver = uc.Chrome(options=options)
#Go to G2 web page so we can accept cookies
driver.get('https://www.g2.com/')
time.sleep(3)
#Accept cookies
driver.find_element_by_xpath('//*[@id="new_user_consent"]/input[6]').click()

#for every company there's in the dataset
for i in range(0,len(df)):
    #use the name of the company to do a query on G2
    driver.get('https://www.g2.com/search?utf8=%E2%9C%93&query=' + df.NAME[i])     

    #if found a captcha, break the loop and print at  which company it stopped to scrap
    if len(driver.find_elements_by_class_name('challenge-form.interactive-form')) ==1:
        print('Stopped at : ', df.NAME[i], 'Index: ',i)
        break
    #if the search result is 0 (meaning G2 has no info on that company), then append "No info" for this company    
    elif len(driver.find_elements_by_class_name('product-listing__head')) == 0 :
        g2_web = 'No info'
        rating = 'No info'
        review ='No info'
        product_description = 'No info'
        alterna = 'No info'
        time.sleep(3)
              
    else:
        #if there's at least one result, then loop over clicking on each of the results and compare the website listed to
        #the one in the dataset to see if we're dealing with the same company
        for found in range(0,len(driver.find_elements_by_class_name('d-ib.c-midnight-100.js-log-click'))):
            driver.find_elements_by_class_name('d-ib.c-midnight-100.js-log-click')[found].click()
            
            try:
                element1 = driver.find_element_by_class_name('p-1.border-top')
            except:
                element1 = driver.find_element_by_class_name('p-1')
            if element1.find_element_by_class_name('link').get_attribute('href')  != df.WEBSITE[i]:
                driver.execute_script("window.history.go(-1)")
                
                #if none of the first 5 results from the query has the same webpage from the company in the dataset, 
                #then append "no info"
                if (found+1 == len(driver.find_elements_by_class_name('d-ib.c-midnight-100.js-log-click'))) or (found == 4):
                    g2_web = 'No info'
                    rating = 'No info'
                    review = 'No info'
                    product_description = 'No info'
                    alterna = 'No info'
                    time.sleep(3)
            #if there's a hit (same website in the dataset and in G2 listing) proceed to get info
            else:
                #if this company has reviews
                g2_web = driver.current_url
                if len(driver.find_elements_by_class_name('f-3')) > 0:
                    five_stars = int(driver.find_elements_by_class_name('f-3')[0].text)
                    four_stars = int(driver.find_elements_by_class_name('f-3')[1].text)
                    three_stars = int(driver.find_elements_by_class_name('f-3')[2].text)
                    two_stars = int(driver.find_elements_by_class_name('f-3')[3].text)
                    one_stars = int(driver.find_elements_by_class_name('f-3')[4].text)

                    rating = ((five_stars * 5) + (four_stars * 4) + (three_stars * 3) + (two_stars * 2) + one_stars) / (five_stars + four_stars + three_stars + two_stars + one_stars)
                    review = five_stars + four_stars + three_stars + two_stars + one_stars
                else:
                    #if there's no review yet on this company, just append "no info"
                    rating = 'No info'
                    review = 'No info'
                #get the elements equivalent to the info needed
                product_description = driver.find_elements_by_class_name('ws-pw')[0].text
                alterna = []
                for alternative in range(0,5):
                    alterna.append(driver.find_elements_by_class_name('module-heading.row-list-item-title.gray.ellipsis')[alternative].text)
                break
    #append the info obtained from the company to the list
    g2_webs.append(g2_web)
    ratings.append(rating)
    reviews.append(review)
    product_descriptions.append(product_description)
    alternatives.append(alterna)
    time.sleep(3)



Stopped at :  Spaceos Index:  11


#### We add the results scraped to the original dataset. Neverhteless, G2 kept blocking the bot, so I figured it wasnt worth to save results. 

In [ ]:
df['G2 Web'] = g2_webs
df['Rating'] = ratings
df['N Reviews'] = reviews
df['Description'] = product_descriptions
df['Alternatives'] = alternatives

df.to_csv('Complete_Dataset.csv', index=False)

#### It would be possible to scrap the entire dataset keeping the results we can get into a short dataset, and resume in big intervals of time within the last result found, but it would take a long time.

In [8]:
df.loc[11]

id                                                      1786924
NAME                                                    Spaceos
WEBSITE                                     https://spaceos.io/
TAGLINE       Offers a smart, simple and intuitive web and m...
ADDRESS       Krakowskie Przedmieście 13, 00-071 Warszawa, P...
INDUSTRIES                                          real estate
Name: 11, dtype: object